In [2]:
# Importing needed libraries
import warnings
warnings.filterwarnings('ignore')

import os
import random
import shutil
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [3]:
# to load the data from Kaggle and having a copy of it that is editable to remove corrupted images

source_data_path = '/kaggle/input/garbage-classification-v2/garbage-dataset'
distination_Data_path = '/kaggle/working/garbage-dataset'

shutil.copytree(source_data_path, distination_Data_path)

'/kaggle/working/garbage-dataset'

In [4]:
# splitting data to training, val, and test
from pathlib import Path
import random

old_dir = Path(distination_Data_path)
new_dir = Path('/kaggle/working/garbage-dataset-splitted')
trainval_dir = new_dir / 'training_and_validation'
test_dir = new_dir / 'test'

test_ratio = 0.1
random.seed(42)

for current_class in old_dir.iterdir():
    if current_class.is_dir():
        images = list(current_class.glob('*.*'))
        # random.shuffle(images)

        splitting_idx = int(len(images) * (1 - test_ratio))
        trainval_images = images[:splitting_idx]
        test_images = images[splitting_idx:]

        (trainval_dir / current_class.name).mkdir(parents=True, exist_ok=True)
        (test_dir / current_class.name).mkdir(parents=True, exist_ok=True)

        for img in trainval_images:
            shutil.copy(img, (trainval_dir / current_class.name / img.name))
        for img in test_images:
            shutil.copy(img, (test_dir / current_class.name / img.name))

    print(f"images of class {current_class.name} splitted successfully")

images of class battery splitted successfully
images of class glass splitted successfully
images of class cardboard splitted successfully
images of class trash splitted successfully
images of class clothes splitted successfully
images of class shoes splitted successfully
images of class paper splitted successfully
images of class plastic splitted successfully
images of class metal splitted successfully
images of class biological splitted successfully


In [5]:
# data_path = '/kaggle/input/garbage-classification-v2/garbage-dataset'   # old data path

# NOW data is prepared to be preprocessed in Kaggle
data_dir = "/kaggle/working/garbage-dataset-splitted/training_and_validation"
test_dir = "/kaggle/working/garbage-dataset-splitted/test"

In [6]:
import os

data_root = "/kaggle/working/garbage-dataset-splitted"


for split in ['training_and_validation', 'test']:
    split_path = os.path.join(data_root, split)

    for class_name in os.listdir(split_path):
        class_path = os.path.join(split_path, class_name)

        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)

                try:
                    img_raw = tf.io.read_file(file_path)
                    img_tensor = tf.io.decode_image(img_raw, channels=3, expand_animations=False)

                except Exception as e:
                    os.remove(file_path)
                    print(f"🗑️ Deleted corrupted file: {file_path}")

I0000 00:00:1752665484.237913      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


🗑️ Deleted corrupted file: /kaggle/working/garbage-dataset-splitted/training_and_validation/paper/paper_2784.jpg
🗑️ Deleted corrupted file: /kaggle/working/garbage-dataset-splitted/training_and_validation/paper/paper_3119.jpg
🗑️ Deleted corrupted file: /kaggle/working/garbage-dataset-splitted/test/paper/paper_1678.jpg


In [7]:
# Splitting data
train_ds = tf.keras.preprocessing.image_dataset_from_directory (
    data_dir,
    subset = 'training',
    batch_size = 32,
    image_size = (224,224),
    shuffle = True,
    seed = 123,
    validation_split = 0.2
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory (
    data_dir,
    subset = 'validation',
    batch_size = 32,
    image_size = (224,224),
    shuffle = True,
    seed = 123,
    validation_split = 0.2
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory (
    test_dir,
    batch_size = 32,
    image_size = (224,224),
    shuffle = False,
    seed = 123
)

Found 17780 files belonging to 10 classes.
Using 14224 files for training.
Found 17780 files belonging to 10 classes.
Using 3556 files for validation.
Found 1979 files belonging to 10 classes.


In [ ]:
#visualising data
class_names = train_ds.class_names
for images, labels in train_ds.take(2):
    plt.figure(figsize=(10, 10))
    for i in range(9):  # Show 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [9]:

from tensorflow.keras.applications.resnet50 import preprocess_input

# preprocessing

#augmentaion
data_augmentation = tf.keras.Sequential([
    layers.RandomBrightness(0.1),
    layers.RandomContrast(0.1),
    layers.RandomFlip('Horizental'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])


def preprocess_train(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

def preprocess_eval(image, label):
    image = preprocess_input(image)
    return image, label


train_ds = train_ds.map(preprocess_train).cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess_eval).cache().prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_eval).cache().prefetch(tf.data.AUTOTUNE)

In [10]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224,224,3)
)

base_model.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [11]:
# Model construction and training

model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,215,818 (100.01 MB)

 Trainable params: 2,628,106 (10.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [12]:
# model compilation and training
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer = Adam(learning_rate=0.001),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)
early_stop = EarlyStopping(patience=3, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 20,
    callbacks=[early_stop]
    )

Epoch 1/20


I0000 00:00:1752665833.219720      95 service.cc:148] XLA service 0x7c43b8003070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752665833.220751      95 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752665834.740873      95 cuda_dnn.cc:529] Loaded cuDNN version 90300


  4/445 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.1966 - loss: 2.9898   

I0000 00:00:1752665839.213984      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


445/445 ━━━━━━━━━━━━━━━━━━━━ 211s 87ms/step - accuracy: 0.7392 - loss: 0.8346 - val_accuracy: 0.9165 - val_loss: 0.2428
Epoch 2/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.8913 - loss: 0.3249 - val_accuracy: 0.9300 - val_loss: 0.2138
Epoch 3/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 25s 57ms/step - accuracy: 0.9110 - loss: 0.2684 - val_accuracy: 0.9331 - val_loss: 0.2206
Epoch 4/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.9203 - loss: 0.2397 - val_accuracy: 0.9362 - val_loss: 0.2126
Epoch 5/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.9325 - loss: 0.2045 - val_accuracy: 0.9390 - val_loss: 0.2069
Epoch 6/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.9370 - loss: 0.1903 - val_accuracy: 0.9418 - val_loss: 0.1909
Epoch 7/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 26s 57ms/step - accuracy: 0.9460 - loss: 0.1675 - val_accuracy: 0.9359 - val_loss: 0.2085
Epoch 8/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.9469 - loss: 0.1739 - val_accura

In [13]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc*100:.2f}%")

62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - accuracy: 0.9631 - loss: 0.1273
Test Accuracy: 95.10%


In [14]:
for layer in base_model.layers[-30:]:
    layer.trainable = True


from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 70s 106ms/step - accuracy: 0.9302 - loss: 0.2213 - val_accuracy: 0.9438 - val_loss: 0.1754
Epoch 2/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 36s 80ms/step - accuracy: 0.9641 - loss: 0.1129 - val_accuracy: 0.9466 - val_loss: 0.1698
Epoch 3/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 35s 80ms/step - accuracy: 0.9758 - loss: 0.0742 - val_accuracy: 0.9511 - val_loss: 0.1708
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 36s 80ms/step - accuracy: 0.9831 - loss: 0.0542 - val_accuracy: 0.9530 - val_loss: 0.1693
Epoch 5/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 35s 80ms/step - accuracy: 0.9901 - loss: 0.0368 - val_accuracy: 0.9544 - val_loss: 0.1787
Epoch 6/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 35s 80ms/step - accuracy: 0.9920 - loss: 0.0268 - val_accuracy: 0.9528 - val_loss: 0.1873
Epoch 7/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 35s 80ms/step - accuracy: 0.9941 - loss: 0.0211 - val_accuracy: 0.9570 - val_loss: 0.1832


In [15]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc*100:.2f}%")

62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 0.9664 - loss: 0.1260
Test Accuracy: 95.50%


In [17]:
model.save('ResNet50_model.h5')

In [18]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

model = tf.keras.models.load_model("ResNet50_model.h5")
class_names = ['Battery', 'Biological', 'Cardboard', 'Clothes', 'Glass', 'Metal', 'Paper', 'Plastic', 'Shoes', 'Trash']

def predict(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    processed_img = preprocess_input(img_array)
    prediction = model.predict(processed_img)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction)
    return f"{predicted_class} ({confidence*100:.2f}%)"

gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Garbage Classifier"
).launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://ba2b2fdec6f3a67965.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Created dataset file at: .gradio/flagged/dataset1.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
